In [1]:
from nltk.tokenize import sent_tokenize
import nltk

nltk.download('punkt')

import torch
import pandas as pd
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from transformers import BertForSequenceClassification
from transformers import BertModel, BertConfig
import random
import numpy as np
import operator

from transformers import AutoTokenizer, AutoModel
from transformers import BertForSequenceClassification, BertTokenizer, TextClassificationPipeline
import json
import glob
import os
import csv
import threading
import time
import multiprocessing
from multiprocessing import Process, Manager

[nltk_data] Downloading package punkt to
[nltk_data]     /home/kit/stud/ulvhi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2021-08-05 10:37:03.144107: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
from Pipeline import Pipeline
import Adapters
import Processors
import Sinks
from imrad_classification import BERTClassificationHandler
from imrad_classification import ClassificationHandler
from haystack_copy import ElasticsearchDocumentStore
from queue import *


[nltk_data] Downloading package punkt to
[nltk_data]     /home/kit/stud/ulvhi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
model_path='/pfs/data5/home/kit/stud/ulvhi/ML_Modells/UnpaywallFinetuned_SciBERT002_epoch_100words_batches5.model'
document_store = ElasticsearchDocumentStore(host="aifb-ls3-vm1.aifb.kit.edu", username="", password="", index="document", duplicate_documents= 'overwrite') #update ElasticsearchSink documents = true
e_adapt = Adapters.ElasticsearchAdapter(document_store, batch_size=10, filters={'_status':["1"]})

/home/kit/stud/ulvhi/.local/lib/python3.8/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.13/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


Load Model and create Elasticsearch adapter

In [25]:
model_path='/pfs/data5/home/kit/stud/ulvhi/ML_Modells/UnpaywallFinetuned_SciBERT002_epoch_100words_batches5.model'
document_store = ElasticsearchDocumentStore(host="aifb-ls3-vm1.aifb.kit.edu", username="", password="", index="document", duplicate_documents= 'overwrite') #update ElasticsearchSink documents = true
e_adapt = Adapters.ElasticsearchAdapter(document_store, batch_size=10, filters={'_status':["4"]})
print(len(e_adapt))


6404824


In [ ]:
STa

In [17]:
update_pipeline = Pipeline(e_adapt, batch_size=100, cpus=1, max_runtime=12000)
#lockArray definieren
update_pipeline.add_processor(Processors.IMRaDClassification(model_path))
update_pipeline.add_sink(Sinks.ElasticsearchSink(document_store))
#update_pipeline.add_sink(Sinks.CSVMetadataSink("testcsv.csv",["arixive-id", "_split_id"])) #

print(update_pipeline)

Some weights of the model checkpoint at allenai/scibert_scivocab_cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were

(ElasticsearchAdapter) ---> *IMRaDClassification*
===> |_ElasticsearchSink_|


In [18]:
len(e_adapt)


15457

In [15]:
update_pipeline.process()

Processing batches: [############################################################] 773/773 Processed documents: 77241 Output Documents: 77241
